# Analisis de datos con python para datos sobre las rocas lunares

## Carga de datos

In [2]:
import pandas as pd

rock_samples = pd.read_csv('./data/rocksamples.csv')



In [3]:
rock_samples.head(10)

,ID,Mission,Type,Subtype,Weight (g),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31
5,10008,Apollo11,Soil,Unsieved,89.0,5.75
6,10009,Apollo11,Breccia,Regolith,112.0,97.27
7,10010,Apollo11,Soil,Unsieved,491.0,91.03
8,10011,Apollo11,Soil,Unsieved,82.6,62.01
9,10014,Apollo11,Soil,Unsieved,50.0,0.00


In [4]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


A partir de esta salida, se ve que durante las misiones Apolo se han obtenido 2229 muestras. Si examina una muestra de los datos, verá que cada fila contiene lo siguiente:

ID: el identificador único que la NASA usa para realizar el seguimiento de la muestra.

Mission: la misión responsable de recuperar la muestra.

Type: el tipo de muestra (tipo de roca u otra clasificación).

Subtype: una clasificación de tipo más específica.

Peso (g): peso original de la muestra en gramos.

Pureza (%): porcentaje que queda de la muestra (parte de la muestra se usa durante la investigación).


## Limpieza de los datos

En este caso, la información de mayor disponibilidad pública que podría afectar a la cantidad de muestras que se pueden obtener es el tipo de nave empleado.


### Conversion del peso de la muestra
Aunque los detalles del diseño de los cohetes son propietarios, parte de la información está disponible de forma pública, como el peso de los módulos (partes del cohete) que transportarán las muestras de vuelta a la Tierra y el peso total que el cohete puede levantar por encima de la atmósfera.

In [5]:
rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x:x*0.001)
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight (kg),Pristine (%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


Aquí primero se han modificado los valores de la columna Peso (g) para que tengan el mismo valor multiplicado por 0,001. Después, se ha cambiado el nombre de la columna por Peso (kg) para que sea más preciso.

### Creacion del dataframe
Pandas, la biblioteca de Python que se va a usar para realizar el análisis de datos, tiene una estructura conocida como "dataframe", la cual resulta realmente eficaz para representar datos bidimensionales. Es posible que haya reconocido que, al ejecutar el código de rock_samples.head(), lo que se imprime es muy similar a una hoja de cálculo de Excel, una excelente manera de describir los elementos dataframe en pandas.

El elemento dataframe rock_samples tiene una fila para cada muestra que se ha obtenido. A pesar de esto, tal como se ha mencionado antes, el objetivo es comprender las muestras de rocas en su totalidad en relación a los cohetes concretos en los que se han transportado.

Crearemos un elemento dataframe con el nombre missions; será un resumen de los datos de cada una de las seis misiones Apolo en las que se han traído muestras. En este elemento dataframe, cree una columna con el nombre Mission, con una fila para cada misión.

In [6]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [7]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


### Suma total del peso de las muestras por mision
Ahora se puede agregar una columna nueva a missions para representar la suma de todas las muestras obtenidas durante esa misión.


In [8]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)
missions

,Mission,Sample weight (kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


Ahora se desglosará este código. La primera línea es sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum(), que se puede dividir de esta forma:

* rock_samples.groupby('Mission'): agrupa todas las filas por los valores de la columna rock_samples.groupby('Mission').

* rock_samples.groupby('Mission')['Weight (kg)']: toma todos los valores de la columna rock_samples.groupby('Mission')['Weight (kg)'], pero los agrupa por valores únicos de la columna Misión.

* rock_samples.groupby('Mission')['Weight (kg)'].sum(): suma todos los valores de la columna rock_samples.groupby('Mission')['Weight (kg)'].sum() para cada valor único de la columna Misión.

Si imprimiera esa línea, obtendría una serie de pandas, que básicamente es un tipo de datos unidimensional o una lista. En la lista, el índice sería el valor único de la columna Mission, en lugar de un número:

In [9]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
sample_total_weight

Mission
Apollo11     21.55424
Apollo12     34.34238
Apollo14     41.83363
Apollo15     75.39910
Apollo16     92.46262
Apollo17    109.44402
Name: Weight (kg), dtype: float64

La línea siguiente, pd.merge(missions, sample_total_weight, on='Mission'), se puede describir de esta forma:

Combine el elemento dataframe missions con la serie sample_total_weight y use la columna missions como el índice sobre el que realizar la combinación. El equipo básicamente hará lo siguiente: para cada valor de la columna Misión del elemento dataframe , se buscará el mismo valor en la serie sample_total_weight y se agregará el valor de la serie a la fila como una nueva columna en el elemento dataframe.

Este ejemplo es bastante sencillo, ya que solo hay seis filas. Por tanto, se puede confirmar que el número 21,55424, por ejemplo, se ha agregado a la fila Apolo 11 en el elemento dataframe missions.

La siguiente línea simplemente cambia el nombre de la columna, como antes, para asegurarse de que los datos son específicos.

La última línea imprime el elemento dataframe missions completo. Como solo hay seis misiones, se puede imprimir el elemento dataframe completo y seguir entendiendo lo que se examina. No es necesario usar head() para imprimir solo las cinco primeras filas.

### Obtencion de la diferencia de pesos entre misiones
Como no somos expertos en cohetes, es importante examinar una gran cantidad de secciones diferentes de los datos disponibles. En este caso, se ve que el peso total de las muestras ha aumentado en cada misión, pero es difícil ver cuánto a simple vista. Se puede agregar una columna más al elemento dataframe missions que simplemente obtenga la diferencia entre la fila actual y la anterior:

In [10]:
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


Es necesario observar que en la primera fila, para Apolo 11, el valor de la columna Diferencia de peso es NaN Esto se denomina valor NULL. Como Apolo11 fue la primera misión, no hay ninguna diferencia entre el peso de la roca obtenida durante Apolo11 y el de la misión anterior. Este valor NaN se puede rellenar con 0:

In [11]:
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)
missions

,Mission,Sample weight (kg),Weight diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140
